# Create Basic Setup
1. Installation
* connect to own drive
* get and install own repo
* get and install waymo repo
2. Data
* transfer waymo dataset from gcs to gdrive
3. Training
* training loop
* visual assessment

Remarks:
* for waymo to work make sure tf is version 1.x
* for tensorboard to work enable 3rd party cookies in your browser

# Installation

In [0]:
'''
MOUNT
'''

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
'''
GET OWN REPO
'''

%cd /content/drive/My\ Drive/Colab\ Notebooks/DeepCV_Packages
!rm -rf /content/drive/My\ Drive/Colab\ Notebooks/DeepCV_Packages/DeepCVLab
!git clone https://github.com/pmcgrath249/DeepCVLab.git

In [0]:
'''
INSTALL EVERYTHING
'''

# permanently change dir 
%cd /content/drive/My\ Drive/Colab\ Notebooks/DeepCV_Packages/DeepCVLab/deepcvlab

# install waymo dataset utils in utils; https://github.com/waymo-research/waymo-open-dataset/blob/master/tutorial/tutorial.ipynb
!cd utils && rm -rf waymo-od > /dev/null
!cd utils && git clone https://github.com/waymo-research/waymo-open-dataset.git waymo-od
!cd utils/waymo-od && git branch -a
!cd utils/waymo-od && git checkout remotes/origin/r1.0
!pip3 install --upgrade pip
!pip3 install waymo-open-dataset

# install requirements
!cd /content/drive/My\ Drive/Colab\ Notebooks/DeepCV_Packages/DeepCVLab/ && pip3 install -r requirements.txt

# install own package
!cd /content/drive/My\ Drive/Colab\ Notebooks/DeepCV_Packages/DeepCVLab/ && python3 -m pip install .

# DATA

Source 1: https://medium.com/@philipplies/transferring-data-from-google-drive-to-google-cloud-storage-using-google-colab-96e088a8c041  

changed because could not find project_id nessecary for this approach 

Source 2: https://cloud.google.com/storage/docs/access-public-data?hl=de 

REMARK: There arise no costs as the bucket is managed by waymo

DATA SOURCE: https://console.cloud.google.com/storage/browser/waymo_open_dataset_v_1_0_0

In [ ]:
'''
DATA FOR TESTING
'''

# copy data such that can remove whole repo without loosing the data
!cp -r /content/drive/My\ Drive/Colab\ Notebooks/DeepCV_Packages/data /content/drive/My\ Drive/Colab\ Notebooks/DeepCV_Packages/DeepCVLab/deepcvlab/data

In [ ]:
'''
AUTHENTICATE GCS
'''

from google.colab import auth
auth.authenticate_user()

In [ ]:
'''
LIST DIR TO BE COPIED
'''

bucket_name = 'waymo_open_dataset_v_1_0_0'
!gsutil ls -r gs://{bucket_name}/

In [ ]:
'''
COPY GCS TO DRIVE
'''

import os

# naming
bucket_name = 'waymo_open_dataset_v_1_0_0'
training_bucket = os.path.join(bucket_name, 'training')

for i in range(32):                                                                                           # from ls above
    dataset_name = 'training_000{}.tar'.format(i) if i < 10 else 'training_00{}.tar'.format(i)                  # right amount of leading zeros
    data_bucket = os.path.join(training_bucket, dataset_name)                 
    !gsutil -m cp -r gs://{data_bucket}/ /content/drive/My\ Drive/Colab\ Notebooks/DeepCV_Packages/data         # copy multi-threaded and recursively

In [ ]:
'''
UNPACKING
'''

import os
%cd /content/drive/My\ Drive/Colab\ Notebooks/DeepCV_Packages/data
file_names = os.listdir()
for file_name in file_names:
  !tar -xvf {file_name}

In [ ]:
'''
OFFLINE DATA CONVERSION
'''

%cd /content/drive/My\ Drive/Colab\ Notebooks/DeepCV_Packages/DeepCVLab/
from deepcvlab.utils import Dense_U_Net_lidar_helper as utils
utils.waymo_to_pytorch_offline()
utils.distribute_data_into_train_val_test([0.6,0.2,0.2])

# Training

In [ ]:
'''
TRAINING WITH TENSORBOARD VISUALIZATION
'''

# import
%cd /content/drive/My\ Drive/Colab\ Notebooks/DeepCV_Packages/DeepCVLab/
from deepcvlab.utils.Dense_U_Net_lidar_helper import get_config
from deepcvlab.agents.Dense_U_Net_lidar_Agent import Dense_U_Net_lidar_Agent as Dense_U_Agent
import os
from pathlib import Path

config = get_config()
config.agent.max_epoch = 100

# if first time training create summary dir before running for tensorboard to work
Path(config.dir.summary).mkdir(exist_ok=True)

# use tensorboard to visualize
%load_ext tensorboard
%tensorboard --logdir {os.path.join(*config.dir.summary.split('/')[-2:])}

# start training
agent = Dense_U_Agent(config=config, torchvision_init=True)
agent.run()
agent.finalize()

In [ ]:
'''
VISUALLY ASSESS DATA AFTER FORWARD PASS
'''

import numpy as np
import matplotlib.pyplot as plt
from deepcvlab.utils.Dense_U_Net_lidar_helper import get_config
from deepcvlab.agents.Dense_U_Net_lidar_Agent import Dense_U_Net_lidar_Agent as Dense_U_Agent

def visual_assessment(img, pred, gt):
  
    num_plots = gt.shape[0]
    fig=plt.figure(figsize=(3*7,num_plots*7))
    for i in range(num_plots):
    
        im = img[i].permute(1, 2, 0).detach().numpy().astype(np.uint8)
        fig.add_subplot(num_plots, 3, i*3+1)   
        plt.imshow(im)

        p = pred[i].permute(1, 2, 0)[:,:,0].detach().numpy().astype(np.uint8)
        fig.add_subplot(num_plots, 3, i*3+2)   
        plt.imshow(p, cmap=plt.cm.gray)

        g = gt[i].permute(1, 2, 0)[:,:,0].detach().numpy()
        fig.add_subplot(num_plots, 3, i*3+3)   
        plt.imshow(g, cmap=plt.cm.gray)

    plt.show()

config = get_config()
config.optimizer.mode = 'train'
agent = Dense_U_Agent(torchvision_init=True)

for image, lidar, _, ht_map in agent.data_loader.train_loader:
            
    if agent.cuda:
        image = image.cuda()
        lidar = lidar.cuda()

    prediction = agent.model(image, lidar)

    visual_assessment(image.cpu(), prediction.cpu(), ht_map)
    continue